In [71]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [141]:
coach_features = pd.read_csv('data/generated_data/coach_features.csv')
stationary_probs_all = pd.read_csv('data/generated_data/stationary_probabilities.csv')
stationary_probs = stationary_probs_all[stationary_probs_all['Season'] >= 2003]
team_season_features = pd.read_csv('data/generated_data/team_season_features.csv')
additional_team_features = pd.read_csv('data/generated_data/additional_team_features.csv')

In [142]:
coach_features.head()

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
0,1102,2003,air-force,False,False,False,0.341,0,0,0,0
1,1103,2003,akron,False,False,False,0.462,0,0,0,0
2,1104,2003,alabama,False,True,False,0.660,4,0,0,0
3,1105,2003,alabama-am,False,False,False,0.518,0,0,0,0
4,1412,2003,alabama-birmingham,False,False,False,0.618,0,0,0,0


In [143]:
stationary_probs.head()

,Unnamed: 0,Season,TeamID,pi_i
5407,5407,2003,1102,0.002807
5408,5408,2003,1103,0.002684
5409,5409,2003,1104,0.004745
5410,5410,2003,1105,0.001239
5411,5411,2003,1106,0.001900


In [144]:
team_season_features.head()

,Unnamed: 0,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3FGM_pg,3_pct,FTM_pg,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement
0,0,2003,1102,28,12,16,0.428571,57.250000,57.000000,0.250000,15.878163,3.0,4.0,0.0,2.0,20.821429,7.821429,0.375643,11.142857,17.107143,0.651357,11.428571,52.232914,109.273561,109.522922,0.515214,-0.086643,None,1.0,0.0,7.0,9.0,8.0,158.0,149.0,-9.0,160.0,151.0,-9.0
1,1,2003,1103,27,13,14,0.481481,78.777778,78.148148,0.629630,11.126039,4.0,5.0,4.0,6.0,16.074074,5.444444,0.338710,19.037037,25.851852,0.736390,12.629630,64.917792,116.651118,116.489034,0.527877,-0.046395,None,1.0,1.0,1.0,13.0,2.0,182.0,198.0,16.0,163.0,187.0,24.0
2,2,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,2.0,1.0,2.0,19.857143,6.357143,0.320144,14.857143,20.928571,0.709898,13.285714,63.168152,108.882967,103.099630,0.708513,-0.101370,Y10,5.0,1.0,6.0,5.0,11.0,38.0,27.0,-11.0,33.0,39.0,6.0
3,3,2003,1105,26,7,19,0.269231,71.769231,76.653846,-4.884615,15.552212,2.0,7.0,1.0,7.0,20.769231,7.576923,0.364815,15.423077,21.846154,0.705986,18.653846,71.461497,97.856445,106.384802,0.285796,-0.016565,None,1.0,0.0,3.0,16.0,4.0,313.0,316.0,3.0,307.0,313.0,6.0
4,4,2003,1106,28,13,15,0.464286,63.607143,63.750000,-0.142857,12.374266,5.0,4.0,2.0,5.0,17.642857,6.107143,0.346154,10.642857,16.464286,0.646421,17.035714,64.071390,98.405306,99.544614,0.492199,-0.027913,None,1.0,0.0,4.0,11.0,5.0,248.0,236.0,-12.0,263.0,227.0,-36.0


In [145]:
additional_team_features.head()

,TeamID,Season,school,returning_min_pct,returning_score_pct,avg_height,avg_yr
0,1101,2021,abilene-christian,72.7,74.3,75,1.6
1,1104,2021,alabama,52.7,54.2,78,1.6
2,1111,2021,appalachian-state,50.0,54.9,76,1.5
3,1116,2021,arkansas,18.3,15.3,78,1.7
4,1124,2021,baylor,67.8,71.7,77,2.0


In [146]:
idx = ['Season', 'TeamID']
features_all = team_season_features.merge(stationary_probs, on=idx)\
.merge(coach_features, on=idx)\
.merge(additional_team_features, on=idx)
features_all.head()

,Unnamed: 0_x,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3FGM_pg,3_pct,FTM_pg,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement,Unnamed: 0_y,pi_i,school_x,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,school_y,returning_min_pct,returning_score_pct,avg_height,avg_yr
0,1994,2009,1103,34,22,12,0.647059,66.411765,60.500000,5.911765,14.439259,11.0,3.0,1.0,3.0,21.617647,7.117647,0.329252,14.117647,19.735294,0.715350,13.735294,61.709427,106.496503,96.694036,0.785303,-0.138244,X13,0.0,0.0,3.0,9.0,3.0,99.0,119.0,20.0,91.0,87.0,-4.0,7403,0.003884,akron,False,False,False,0.614,1,0,0,0,akron,47.6,41.7,76,1.3
1,1997,2009,1106,29,20,9,0.689655,69.482759,63.586207,5.896552,13.737301,12.0,1.0,1.0,1.0,17.310345,5.689655,0.328685,12.689655,20.034483,0.633391,13.758621,63.553324,109.728685,100.280228,0.774442,-0.084787,Y16a,0.0,0.0,1.0,8.0,1.0,179.0,221.0,42.0,195.0,209.0,14.0,7406,0.002482,alabama-state,False,False,False,0.520,1,0,0,0,alabama-state,52.3,58.3,76,1.4
2,2000,2009,1110,31,24,7,0.774194,64.451613,58.419355,6.032258,13.994201,11.0,1.0,7.0,0.0,18.838710,7.096774,0.376712,10.645161,16.419355,0.648330,13.225806,58.392843,110.563523,98.285184,0.796879,-0.022685,W14,0.0,0.0,2.0,5.0,2.0,73.0,80.0,7.0,109.0,105.0,-4.0,7409,0.002973,american,False,False,False,0.565,7,2,0,0,american,79.1,84.0,75,2.3
3,2002,2009,1112,32,19,13,0.593750,72.187500,68.437500,3.750000,12.942179,5.0,3.0,3.0,4.0,15.656250,6.187500,0.395210,14.687500,20.000000,0.734375,12.593750,61.335267,116.681775,111.435022,0.677442,-0.083692,Y12,5.0,1.0,9.0,4.0,14.0,62.0,43.0,-19.0,39.0,35.0,-4.0,7411,0.004633,arizona,False,False,False,0.600,1,1,0,0,arizona,53.2,57.7,77,1.4
4,2003,2009,1113,33,24,9,0.727273,69.545455,60.060606,9.484848,14.329713,12.0,3.0,4.0,3.0,22.090909,8.090909,0.366255,13.878788,18.757576,0.739903,11.515152,57.697745,119.251047,102.404720,0.884890,-0.157617,X06,9.0,5.0,5.0,4.0,14.0,31.0,27.0,-4.0,12.0,9.0,-3.0,7412,0.006156,arizona-state,False,True,False,0.603,7,1,0,0,arizona-state,93.0,94.6,77,1.6


In [147]:
features_all.columns

Index(['Unnamed: 0_x', 'Season', 'TeamID', 'total_games', 'num_wins',
       'num_losses', 'win_pct', 'avg_points_for', 'avg_points_against',
       'avg_game_margin', 'std_game_margin', 'non_home_wins', 'home_losses',
       'close_wins', 'close_losses', '3FGA_pg', '3FGM_pg', '3_pct', 'FTM_pg',
       'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'Seed', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'prev_net', 'net_improvement', 'final_pom', 'prev_pom',
       'pom_improvement', 'Unnamed: 0_y', 'pi_i', 'school_x',
       'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'school_y', 'returning_min_pct', 'returning_score_pct', 'avg_height',
       'avg_yr'],
      dtype='object')

In [148]:
features = features_all[['Season', 'TeamID', 'total_games', 'win_pct', 'avg_points_for', 
              'avg_points_against', 'avg_game_margin', 'std_game_margin', 
              'non_home_wins', 'home_losses', 'close_wins', 'close_losses', 
              '3FGA_pg', '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 
              'off_eff', 'def_eff', 'pythag_win_pct', 'luck', 'Seed', 'wins_vs_tourney_teams',
              'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
              'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
              'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
              'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
              'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
              'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr']]

In [149]:
features.head()

,Season,TeamID,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3_pct,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,net_improvement,final_pom,pom_improvement,pi_i,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,avg_height,avg_yr
0,2009,1103,34,0.647059,66.411765,60.500000,5.911765,14.439259,11.0,3.0,1.0,3.0,21.617647,0.329252,19.735294,0.715350,13.735294,61.709427,106.496503,96.694036,0.785303,-0.138244,X13,0.0,0.0,3.0,9.0,3.0,99.0,20.0,91.0,-4.0,0.003884,False,False,False,0.614,1,0,0,0,47.6,41.7,76,1.3
1,2009,1106,29,0.689655,69.482759,63.586207,5.896552,13.737301,12.0,1.0,1.0,1.0,17.310345,0.328685,20.034483,0.633391,13.758621,63.553324,109.728685,100.280228,0.774442,-0.084787,Y16a,0.0,0.0,1.0,8.0,1.0,179.0,42.0,195.0,14.0,0.002482,False,False,False,0.520,1,0,0,0,52.3,58.3,76,1.4
2,2009,1110,31,0.774194,64.451613,58.419355,6.032258,13.994201,11.0,1.0,7.0,0.0,18.838710,0.376712,16.419355,0.648330,13.225806,58.392843,110.563523,98.285184,0.796879,-0.022685,W14,0.0,0.0,2.0,5.0,2.0,73.0,7.0,109.0,-4.0,0.002973,False,False,False,0.565,7,2,0,0,79.1,84.0,75,2.3
3,2009,1112,32,0.593750,72.187500,68.437500,3.750000,12.942179,5.0,3.0,3.0,4.0,15.656250,0.395210,20.000000,0.734375,12.593750,61.335267,116.681775,111.435022,0.677442,-0.083692,Y12,5.0,1.0,9.0,4.0,14.0,62.0,-19.0,39.0,-4.0,0.004633,False,False,False,0.600,1,1,0,0,53.2,57.7,77,1.4
4,2009,1113,33,0.727273,69.545455,60.060606,9.484848,14.329713,12.0,3.0,4.0,3.0,22.090909,0.366255,18.757576,0.739903,11.515152,57.697745,119.251047,102.404720,0.884890,-0.157617,X06,9.0,5.0,5.0,4.0,14.0,31.0,-4.0,12.0,-3.0,0.006156,False,True,False,0.603,7,1,0,0,93.0,94.6,77,1.6


## Tourney games

In [150]:
tourney_games_all = pd.read_csv('data/kaggle_data/MNCAATourneyCompactResults.csv')
tourney_games = tourney_games_all[tourney_games_all['Season'] >= 2003]
tourney_games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1136,2003,134,1421,92,1411,84,N,1
1137,2003,136,1112,80,1436,51,N,0
1138,2003,136,1113,84,1272,71,N,0
1139,2003,136,1141,79,1166,73,N,0
1140,2003,136,1143,76,1301,74,N,1


In [151]:
tourney_games['rand'] = [random.random() for i in range(len(tourney_games))]
tourney_games['teams'] = [(t1, t2) if r >= 0.5 else (t2, t1) for t1, t2, r in \
                          zip(tourney_games['WTeamID'], tourney_games['LTeamID'], tourney_games['rand'])]
tourney_games['t1_score_diff'] = [t1-t2 if r >= 0.5 else t2-t1 for t1, t2, r in \
                          zip(tourney_games['WScore'], tourney_games['LScore'], tourney_games['rand'])]
tourney_games[['team1','team2']] = pd.DataFrame(tourney_games['teams'].tolist(), index=tourney_games.index)

In [152]:
tourney_games

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,rand,teams,t1_score_diff,team1,team2
1136,2003,134,1421,92,1411,84,N,1,0.201800,"(1411, 1421)",-8,1411,1421
1137,2003,136,1112,80,1436,51,N,0,0.158153,"(1436, 1112)",-29,1436,1112
1138,2003,136,1113,84,1272,71,N,0,0.975257,"(1113, 1272)",13,1113,1272
1139,2003,136,1141,79,1166,73,N,0,0.456158,"(1166, 1141)",-6,1166,1141
1140,2003,136,1143,76,1301,74,N,1,0.411129,"(1301, 1143)",-2,1301,1143
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,2021,148,1211,85,1425,66,N,0,0.851569,"(1211, 1425)",19,1211,1425
2313,2021,148,1417,51,1276,49,N,0,0.708532,"(1417, 1276)",2,1417,1276
2314,2021,152,1124,78,1222,59,N,0,0.148009,"(1222, 1124)",-19,1222,1124
2315,2021,152,1211,93,1417,90,N,1,0.360538,"(1417, 1211)",-3,1417,1211


In [153]:
tourney_results = tourney_games[['Season', 'team1', 'team2', 't1_score_diff']]

In [154]:
tourney_results

,Season,team1,team2,t1_score_diff
1136,2003,1411,1421,-8
1137,2003,1436,1112,-29
1138,2003,1113,1272,13
1139,2003,1166,1141,-6
1140,2003,1301,1143,-2
...,...,...,...,...
2312,2021,1211,1425,19
2313,2021,1417,1276,2
2314,2021,1222,1124,-19
2315,2021,1417,1211,-3


In [155]:
matchup_features_all = tourney_results.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])\
.merge(features, left_on=['Season', 'team2'], right_on=['Season', 'TeamID'], suffixes=['_1', '_2'])
matchup_features_all.head()

,Season,team1,team2,t1_score_diff,TeamID_1,total_games_1,win_pct_1,avg_points_for_1,avg_points_against_1,avg_game_margin_1,std_game_margin_1,non_home_wins_1,home_losses_1,close_wins_1,close_losses_1,3FGA_pg_1,3_pct_1,FTA_pg_1,FT_pct_1,TOV_pg_1,avg_pace_1,off_eff_1,def_eff_1,pythag_win_pct_1,luck_1,Seed_1,wins_vs_tourney_teams_1,away_wins_vs_tourney_teams_1,losses_vs_tourney_teams_1,losses_vs_non_tourney_teams_1,games_vs_tourney_teams_1,final_net_1,net_improvement_1,final_pom_1,pom_improvement_1,pi_i_1,is_ap_pre_top_5_1,is_ap_pre_top_15_1,is_ap_pre_top_25_1,coach_WL_car_1,tourneys_car_1,sw16_car_1,ff_car_1,champ_car_1,returning_min_pct_1,returning_score_pct_1,avg_height_1,avg_yr_1,TeamID_2,total_games_2,win_pct_2,avg_points_for_2,avg_points_against_2,avg_game_margin_2,std_game_margin_2,non_home_wins_2,home_losses_2,close_wins_2,close_losses_2,3FGA_pg_2,3_pct_2,FTA_pg_2,FT_pct_2,TOV_pg_2,avg_pace_2,off_eff_2,def_eff_2,pythag_win_pct_2,luck_2,Seed_2,wins_vs_tourney_teams_2,away_wins_vs_tourney_teams_2,losses_vs_tourney_teams_2,losses_vs_non_tourney_teams_2,games_vs_tourney_teams_2,final_net_2,net_improvement_2,final_pom_2,pom_improvement_2,pi_i_2,is_ap_pre_top_5_2,is_ap_pre_top_15_2,is_ap_pre_top_25_2,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,avg_height_2,avg_yr_2
0,2009,1106,1287,-15,1106,29,0.689655,69.482759,63.586207,5.896552,13.737301,12.0,1.0,1.0,1.0,17.310345,0.328685,20.034483,0.633391,13.758621,63.553324,109.728685,100.280228,0.774442,-0.084787,Y16a,0.0,0.0,1.0,8.0,1.0,179.0,42.0,195.0,14.0,0.002482,False,False,False,0.520,1,0,0,0,52.3,58.3,76,1.4,1287,32,0.531250,69.343750,68.687500,0.656250,12.758843,7.0,1.0,2.0,3.0,15.562500,0.327309,23.218750,0.724092,15.406250,63.520320,109.045115,108.066128,0.533019,-0.001769,Y16b,1.0,1.0,1.0,14.0,2.0,141.0,25.0,167.0,21.0,0.002969,False,False,False,0.490,1,0,0,0,63.6,71.1,76,0.9
1,2009,1151,1163,-56,1151,32,0.500000,75.000000,76.468750,-1.468750,15.060263,5.0,3.0,2.0,3.0,23.250000,0.349462,26.687500,0.651054,15.406250,67.625733,110.510353,111.322263,0.432963,0.067037,Z16,0.0,0.0,5.0,11.0,5.0,174.0,11.0,213.0,-24.0,0.002351,False,False,False,0.556,2,0,0,0,60.0,60.9,77,1.9,1163,31,0.870968,77.322581,63.967742,13.354839,13.284841,13.0,2.0,1.0,1.0,13.677419,0.346698,26.290323,0.677301,12.677419,64.244714,118.162195,95.598451,0.933232,-0.062265,Z01,8.0,5.0,3.0,1.0,11.0,8.0,-5.0,3.0,1.0,0.007124,True,False,False,0.702,21,12,3,2,88.3,89.9,78,2.0
2,2009,1345,1163,-12,1345,34,0.735294,69.352941,59.117647,10.235294,14.834496,10.0,3.0,2.0,5.0,19.117647,0.350769,18.558824,0.700475,11.647059,62.379482,110.234427,94.091258,0.902132,-0.166837,Z05,9.0,5.0,7.0,2.0,16.0,20.0,8.0,14.0,3.0,0.006030,False,True,False,0.663,4,1,0,0,81.9,82.2,77,1.3,1163,31,0.870968,77.322581,63.967742,13.354839,13.284841,13.0,2.0,1.0,1.0,13.677419,0.346698,26.290323,0.677301,12.677419,64.244714,118.162195,95.598451,0.933232,-0.062265,Z01,8.0,5.0,3.0,1.0,11.0,8.0,-5.0,3.0,1.0,0.007124,True,False,False,0.702,21,12,3,2,88.3,89.9,78,2.0
3,2009,1281,1163,-7,1281,34,0.823529,81.117647,66.470588,14.647059,17.993271,10.0,0.0,2.0,0.0,19.735294,0.357675,22.794118,0.668387,12.352941,69.022871,117.555364,96.019546,0.941036,-0.117506,Z03,8.0,4.0,4.0,2.0,12.0,10.0,4.0,10.0,-2.0,0.006957,False,False,False,0.670,4,2,0,0,50.2,56.0,78,1.7,1163,31,0.870968,77.322581,63.967742,13.354839,13.284841,13.0,2.0,1.0,1.0,13.677419,0.346698,26.290323,0.677301,12.677419,64.244714,118.162195,95.598451,0.933232,-0.062265,Z01,8.0,5.0,3.0,1.0,11.0,8.0,-5.0,3.0,1.0,0.007124,True,False,False,0.702,21,12,3,2,88.3,89.9,78,2.0
4,2009,1401,1163,-26,1401,32,0.718750,71.718750,66.343750,5.375000,10.412583,7.0,2.0,3.0,0.0,16.500000,0.365530,25.625000,0.695122,12.843750,62.686067,114.560752,104.752982,0.747180,-0.028430,Z09,6.0,1.0,5.0,4.0,11.0,36.0,16.0,52.0,10.0,0.004272,False,False,False,0.591,3,1,0,0,60.0,59.9,77,1.7,1163,31,0.870968,77.322581,63.967742,13.354839,13.284841,13.0,2.0,1.0,1.0,13.67741

In [156]:
features.columns

Index(['Season', 'TeamID', 'total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'Seed', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr'],
      dtype='object')

In [158]:
difference_cols = ['total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr']
boolean_cols = ['is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25']

In [159]:
for col in difference_cols:
    matchup_features_all['diff_'+col] = matchup_features_all[col+'_1'] - matchup_features_all[col+'_2']

In [160]:
pd.set_option("display.max_columns", None)
matchup_features_all.head()

,Season,team1,team2,t1_score_diff,TeamID_1,total_games_1,win_pct_1,avg_points_for_1,avg_points_against_1,avg_game_margin_1,std_game_margin_1,non_home_wins_1,home_losses_1,close_wins_1,close_losses_1,3FGA_pg_1,3_pct_1,FTA_pg_1,FT_pct_1,TOV_pg_1,avg_pace_1,off_eff_1,def_eff_1,pythag_win_pct_1,luck_1,Seed_1,wins_vs_tourney_teams_1,away_wins_vs_tourney_teams_1,losses_vs_tourney_teams_1,losses_vs_non_tourney_teams_1,games_vs_tourney_teams_1,final_net_1,net_improvement_1,final_pom_1,pom_improvement_1,pi_i_1,is_ap_pre_top_5_1,is_ap_pre_top_15_1,is_ap_pre_top_25_1,coach_WL_car_1,tourneys_car_1,sw16_car_1,ff_car_1,champ_car_1,returning_min_pct_1,returning_score_pct_1,avg_height_1,avg_yr_1,TeamID_2,total_games_2,win_pct_2,avg_points_for_2,avg_points_against_2,avg_game_margin_2,std_game_margin_2,non_home_wins_2,home_losses_2,close_wins_2,close_losses_2,3FGA_pg_2,3_pct_2,FTA_pg_2,FT_pct_2,TOV_pg_2,avg_pace_2,off_eff_2,def_eff_2,pythag_win_pct_2,luck_2,Seed_2,wins_vs_tourney_teams_2,away_wins_vs_tourney_teams_2,losses_vs_tourney_teams_2,losses_vs_non_tourney_teams_2,games_vs_tourney_teams_2,final_net_2,net_improvement_2,final_pom_2,pom_improvement_2,pi_i_2,is_ap_pre_top_5_2,is_ap_pre_top_15_2,is_ap_pre_top_25_2,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,avg_height_2,avg_yr_2,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_avg_height,diff_avg_yr
0,2009,1106,1287,-15,1106,29,0.689655,69.482759,63.586207,5.896552,13.737301,12.0,1.0,1.0,1.0,17.310345,0.328685,20.034483,0.633391,13.758621,63.553324,109.728685,100.280228,0.774442,-0.084787,Y16a,0.0,0.0,1.0,8.0,1.0,179.0,42.0,195.0,14.0,0.002482,False,False,False,0.520,1,0,0,0,52.3,58.3,76,1.4,1287,32,0.531250,69.343750,68.687500,0.656250,12.758843,7.0,1.0,2.0,3.0,15.562500,0.327309,23.218750,0.724092,15.406250,63.520320,109.045115,108.066128,0.533019,-0.001769,Y16b,1.0,1.0,1.0,14.0,2.0,141.0,25.0,167.0,21.0,0.002969,False,False,False,0.490,1,0,0,0,63.6,71.1,76,0.9,-3,0.158405,0.139009,-5.101293,5.240302,0.978457,5.0,0.0,-1.0,-2.0,1.747845,0.001376,-3.184267,-0.090701,-1.647629,0.033004,0.683570,-7.785900,0.241424,-0.083019,-1.0,-1.0,0.0,-6.0,-1.0,38.0,17.0,28.0,-7.0,-0.000486,0.030,0,0,0,0,-11.3,-12.8,0,0.5
1,2009,1151,1163,-56,1151,32,0.500000,75.000000,76.468750,-1.468750,15.060263,5.0,3.0,2.0,3.0,23.250000,0.349462,26.687500,0.651054,15.406250,67.625733,110.510353,111.322263,0.432963,0.067037,Z16,0.0,0.0,5.0,11.0,5.0,174.0,11.0,213.0,-24.0,0.002351,False,False,False,0.556,2,0,0,0,60.0,60.9,77,1.9,1163,31,0.870968,77.322581,63.967742,13.354839,13.284841,13.0,2.0,1.0,1.0,13.677419,0.346698,26.290323,0.677301,12.677419,64.244714,118.162195,95.598451,0.933232,-0.062265,Z01,8.0,5.0,3.0,1.0,11.0,8.0,-5.0,3.0,1.0,0.007124,True,False,False,0.702,21,12,3,2,88.3,89.9,78,2.0,1,-0.370968,-2.322581,12.501008,-14.823589,1.775422,-8.0,1.0,1.0,2.0,9.572581,0.002764,0.397177,-0.026247,2.728831,3.381019,-7.651842,15.723813,-0.500269,0.129301,-8.0,-5.0,2.0,10.0,-6.0,166.0,16.0,210.0,-25.0,-0.004773,-0.146,-19,-12,-3,-2,-28.3,-29.0,-1,-0.1
2,2009,1345,1163,-12,1345,34,0.735294,69.352941,59.117647,10.235294,14.834496,10.0,3.0,2.0,5.0,19.117647,0.350769,18.558824,0.700475,11.647059,62.379482,110.234427,94.091258,0.902132,-0.166837,Z05,9.0,5.0,7.0,2.0,16.0,20.0,8.0,14.0,3.0,0.006030,False,True,False,0.663,4,1,0,0,81.9,82.2,77,1.3,1163,31,

In [161]:
diff_cols = [col for col in matchup_features_all.columns if 'diff' in col]
matchup_features_basic = matchup_features_all[diff_cols]
matchup_features_basic.head()

,t1_score_diff,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_avg_height,diff_avg_yr
0,-15,-3,0.158405,0.139009,-5.101293,5.240302,0.978457,5.0,0.0,-1.0,-2.0,1.747845,0.001376,-3.184267,-0.090701,-1.647629,0.033004,0.683570,-7.785900,0.241424,-0.083019,-1.0,-1.0,0.0,-6.0,-1.0,38.0,17.0,28.0,-7.0,-0.000486,0.030,0,0,0,0,-11.3,-12.8,0,0.5
1,-56,1,-0.370968,-2.322581,12.501008,-14.823589,1.775422,-8.0,1.0,1.0,2.0,9.572581,0.002764,0.397177,-0.026247,2.728831,3.381019,-7.651842,15.723813,-0.500269,0.129301,-8.0,-5.0,2.0,10.0,-6.0,166.0,16.0,210.0,-25.0,-0.004773,-0.146,-19,-12,-3,-2,-28.3,-29.0,-1,-0.1
2,-12,3,-0.135674,-7.969639,-4.850095,-3.119545,1.549656,-3.0,1.0,1.0,4.0,5.440228,0.004071,-7.731499,0.023175,-1.030361,-1.865232,-7.927768,-1.507192,-0.031101,-0.104573,1.0,0.0,4.0,1.0,5.0,12.0,13.0,11.0,2.0,-0.001094,-0.039,-17,-11,-3,-2,-6.4,-7.7,-1,-0.7
3,-7,3,-0.047438,3.795066,2.502846,1.292220,4.708430,-3.0,-2.0,1.0,-1.0,6.057875,0.010977,-3.496205,-0.008914,-0.324478,4.778157,-0.606831,0.421095,0.007803,-0.055242,0.0,-1.0,1.0,1.0,1.0,2.0,9.0,7.0,-3.0,-0.000167,-0.032,-17,-10,-3,-2,-38.1,-33.9,0,-0.3
4,-26,1,-0.152218,-5.603831,2.376008,-7.979839,-2.872258,-6.0,0.0,2.0,-1.0,2.822581,0.018832,-0.665323,0.017821,0.166331,-1.558647,-3.601443,9.154531,-0.186053,0.033835,-2.0,-4.0,2.0,3.0,0.0,28.0,21.0,49.0,9.0,-0.002852,-0.111,-18,-11,-3,-2,-28.3,-30.0,-1,-0.3


In [162]:
matchup_features_basic.to_csv('data/model_data/matchup_features_basic.csv', index=False)